### import packages

In [1]:
import os
import numpy as np
import pandas as pd
#from numba import jit
from scipy.stats import norm
import yfinance as yf
import pandas_datareader.data as web
#from rateslib import dt
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
file_dir = os.getcwd()
data_dir = os.path.join(file_dir, 'datasets')

### Pricing Models to be considered:

##### 1. Binomial Trees

##### 2. Black-Scholes Model

##### 3. Heston?

##### 4. Monte Carlo (I have the code)

In [3]:
# use jit to improve compile speed
# @jit(nopython=True)  #thi an error if the function cannot be compiled.

def blackscholes(asset_0, debt, T, r, sigma_asset):
    """
    Price of a European call in the Black-Scholes model - where equity_0 = call price, asset_0 = underlying price, debt = strike price.
    """
    d1 = (np.log(asset_0)-np.log(debt)+(r+sigma_asset**2/2)*T)/(sigma_asset*np.sqrt(T))
    d2 = d1-sigma_asset*np.sqrt(T)

    return asset_0 * norm.cdf(d1) - np.exp(-r*T) * debt * norm.cdf(d2)

In [4]:
def BS_implied_spread(asset_0, debt, T, r, sigma_asset):
    
    leverage = (debt * np.exp(-r * T)) / asset_0 
    d1 = (np.log(asset_0) - np.log(debt) + (r + sigma_asset**2 / 2) * T) / (sigma_asset * np.sqrt(T))
    d2 = d1 - sigma_asset * np.sqrt(T)

    implied_spread = -np.log((norm.cdf(d2) + (norm.cdf(-d1) / leverage)) / T)
    return implied_spread

In [5]:
cds_csv_path = os.path.join(data_dir, 'cds.csv')
cds_data = pd.read_csv(cds_csv_path, index_col='Date')

In [6]:
cds_data.head()

,Ticker,Company,PX1,PX2,PX3,PX4,PX5,PX6,PX7,PX8,PX9,PX10
Date,,,,,,,,,,,,
2015-01-01,A,Agilent Technologies Inc,14.850,25.05,39.055,73.715,86.30,111.305,125.905,144.300,158.600,168.195
2015-01-02,A,Agilent Technologies Inc,14.855,25.15,39.160,73.600,86.40,111.510,126.100,144.375,158.775,168.375
2015-01-05,A,Agilent Technologies Inc,13.455,24.96,39.310,70.200,88.26,111.970,125.060,143.165,157.385,166.825
2015-01-06,A,Agilent Technologies Inc,15.535,24.88,39.300,72.680,91.10,116.610,130.890,150.365,165.655,175.885
2015-01-07,A,Agilent Technologies Inc,15.215,25.50,40.270,73.880,92.50,117.780,131.940,151.145,166.215,176.295


In [7]:
cds_apple = cds_data[cds_data['Ticker'] == 'AAPL']
equity_apple = yf.download('AAPL', start='2015-01-01', end='2021-09-11')['Adj Close']

[*********************100%***********************]  1 of 1 completed


In [8]:
csv_path_apple = os.path.join(data_dir, 'aapl_data.csv')
apple_data = pd.read_csv(csv_path_apple, index_col='apdedateq')

ValueError: Index apdedateq invalid

In [9]:
apple_data.index = pd.to_datetime(apple_data.index)
cds_apple.index = pd.to_datetime(cds_apple.index)

NameError: name 'apple_data' is not defined

In [10]:
apple_data['asset_val'] = apple_data['atq']
apple_data['debt_val'] = apple_data['ltq']

NameError: name 'apple_data' is not defined

In [11]:
risk_free_rate = web.DataReader('TB3MS', 'fred', start='2015-01-01', end='2021-09-11')

In [12]:
apple_data['asset_returns'] = (np.log(apple_data['asset_val'] / apple_data['asset_val'].shift(1)))*4
asset_vol_naive = np.std(apple_data['asset_returns'])*np.sqrt(4) # multiply sqrt(4) because asset values reported quarterly

NameError: name 'apple_data' is not defined

In [13]:
testspread = BS_implied_spread(asset_0 = (apple_data['asset_val'].iloc[0] / apple_data['cshoq'].iloc[0]), 
                                            debt=(apple_data['debt_val'].iloc[0] / apple_data['cshoq'].iloc[0]), 
                                            T=1, r=risk_free_rate.iloc[0].item(), sigma_asset=asset_vol_naive)

NameError: name 'apple_data' is not defined

In [14]:
blackscholes(asset_0=(apple_data['asset_val'].iloc[0] / apple_data['cshoq'].iloc[0]), debt=(apple_data['debt_val'].iloc[0] / apple_data['cshoq'].iloc[0]), T=1, r=risk_free_rate.iloc[0], sigma_asset=asset_vol_naive)

NameError: name 'apple_data' is not defined

In [ ]:
apple_data['risk_free'] = risk_free_rate

In [ ]:
risk_free_rate_reindexed = risk_free_rate.reindex(apple_data.index, method='bfill') # backfill bcs you cant look forward w/ fin data

In [ ]:
apple_spreads_1y = []
for index, row in apple_data.iterrows():
    assetval = row['asset_val'] / row['cshoq']
    debtval = row['debt_val'] / row['cshoq']
    T_1y = 1
    sigma = asset_vol_naive
    rf = risk_free_rate_reindexed.loc[index].item()

    # try:
    #     rf = risk_free_rate.loc[index]
    # except KeyError:
    #     rf = risk_free_rate_reindexed.iloc[risk_free_rate.index.get_loc(index, method='nearest')]

    implied_spread_1y = BS_implied_spread(asset_0=assetval, debt=debtval, T=T_1y, r=rf, sigma_asset=sigma)
    apple_spreads_1y.append(implied_spread_1y)

In [ ]:
type(cds_apple.index)

In [ ]:
new_spreads_apple = [] 
for i in range(len(apple_spreads_1y)):
    new_spreads_apple.append(apple_spreads_1y[i]*100)

In [ ]:
plt.plot(apple_data.index, new_spreads_apple)
plt.plot(cds_apple.index, cds_apple['PX1'])

In [ ]:
tickers = cds_data['Ticker'].unique()
tickers = tickers.tolist()

In [ ]:
company_names = cds_data['Company'].unique()
company_names = company_names.tolist()

In [ ]:
csv_path_apple = os.path.join(data_dir, 'aapl_data.csv')
apple_data = pd.read_csv(csv_path_apple, index_col='apdedateq')

In [ ]:

file_path_comps = os.path.join(data_dir, 'company_list.txt')
with open(file_path_comps, 'w') as file:
    for company in company_names:
        file.write(f"{company}\n")

In [ ]:
cds_data

In [ ]:
tickers

# 

In [ ]:
company_stock_data = {}

for ticker in tickers:
    series = yf.download(ticker, start = '2015-01-01', end='2021-09-11')['Adj Close']
    company_stock_data[ticker] = series

In [ ]:
company_stock_data_df = pd.DataFrame(company_stock_data)

In [ ]:
len(company_stock_data_df['AAPL'])

In [ ]:
for company in company_stock_data_df:
    if len(company_stock_data_df[company].dropna()) < 0.6 * len(company_stock_data_df[company]):
        company_stock_data_df = company_stock_data_df.drop(company, axis=1)

In [ ]:
company_stock_data_df = company_stock_data_df[:-43] # all NA values at the end

In [ ]:
final_ticker_list = company_stock_data_df.columns.tolist()

In [ ]:
len(final_ticker_list)

In [ ]:
file_path_tickers = os.path.join(data_dir, 'tickers.txt')
with open(file_path_tickers, 'w') as file:
    for ticker in final_ticker_list:
        file.write(f"{ticker}\n")

In [ ]:
csv_file_path_stocks = os.path.join(data_dir, 'company_stock_data.csv')
company_stock_data_df.to_csv(csv_file_path_stocks)

In [ ]:
nasdaq_historical = yf.download('^IXIC', start='1990-01-01', end='2024-01-01')
nasdaq_historical['log_returns'] = np.log(nasdaq_historical['Adj Close'] / nasdaq_historical['Adj Close'].shift(1))
nasdaq_historical.head()

In [ ]:
# calc the daily volatility of returns and use as a super naive volatility estimate
daily_vol = np.std(nasdaq_historical['log_returns'].dropna())
annualized_vol_naive = daily_vol * np.sqrt(252)

nasdaq_prices = nasdaq_historical['Adj Close']

In [ ]:
plt.plot(nasdaq_prices.index, nasdaq_prices)
plt.title("Historical Prices of NASDAQ")
plt.axvline(pd.Timestamp('1997-01-01'), color='red', linestyle='--', label='1996-01-02')
plt.axvline(pd.Timestamp('2002-01-02'), color='red', linestyle='--', label='2002-01-02')
plt.xlabel("Time")
plt.ylabel(("Daily Price (Adjusted Closing)"))

In [ ]:
nasdaq_bubble = nasdaq_prices.loc['1997-01-01':'2003-01-01']
nasdaq_bubble_full = pd.DataFrame(nasdaq_bubble)

# just used generic values to show
nasdaq_bubble_full['K'] = nasdaq_bubble_full['Adj Close']
nasdaq_bubble_full['T'] = 1
nasdaq_bubble_full['Rf'] = 0.05 
nasdaq_bubble_full['sigma'] = annualized_vol_naive

In [ ]:
bs_call_prices = []
for index, row in nasdaq_bubble_full.iterrows():
    S0 = row['Adj Close']
    K = row['K']
    T = row['T']
    r = row['Rf']
    sigma = row['sigma']

    bs_price = blackscholes(S0, K, T, r, sigma)
    bs_call_prices.append(bs_price)

In [ ]:
plt.plot(nasdaq_bubble_full.index, bs_call_prices, color='r', label='call price')
plt.plot(nasdaq_bubble_full['Adj Close'], color='b', label='index price')
plt.legend()

In [ ]:
plt.plot(nasdaq_bubble_full.index, bs_call_prices)